# Quickmaths PyRVA
June 23, 2022

## Challenge
You must solve 1000 of these math problems that are outputted in the following format {number} {operation} {number} to get the flag. 
Division is integer division using the // operator. 
The input is being checked through python input() function. 
Good luck! 

## Connect
[Netcat](https://www.mankier.com/1/netcat) is a command line utility that for creating network socket connections.

Every machine on the network is connectable by it's ip address (routing) through ports (application) based on a server/client principle.

Your machine (client) can make a request to a port of a remote machine (server) and if there is an application listening for connections on that port, a socket connection can be established.

*Internet Assigned Numbers Authority* (IANA) maintains a [list](https://www.iana.org/assignments/service-names-port-numbers/service-names-port-numbers.txt) of "well-known" ports, but truthfully, these port assignments are arbitrary and can be re-used whenever the client and server are both in agreement. Generally speaking, you won't know if there is an application listening on a machine at a given port until you try to connect to it. If you get a response, we can assume the port is open and ready for connections.

Today's challenge is being hosted at this address and port:

In [ ]:
nc 44.202.214.112 1228

By using a tool like [American Registry for Internet Numbers (ARIN)](https://search.arin.net/rdap/?query=44.202.214.112), we can discover that this address is owned by Amazon and the prompt tells us it is serving a python script at the available port.


It's going to take you a while to solve that many math problems manually or it might be easier to find a way to automate the process...

## Python solution

#### Create a virtual environment in your working directory.

```
python -m venv venv
source venv/bin/activate
```

#### Import

Feel free to experiment with any set of tools you would like, but one such tool that creates a wrapper for rapid development with socket connections is [PwnTools](https://github.com/Gallopsled/pwntools/blob/dev/examples/remote.py).

In [1]:
from pwn import *

#### Variables

In [2]:
remote_host = "44.202.214.112"
remote_port = 1228

#### Create a remote connection

In [3]:
conn = remote(remote_host,remote_port)

[x] Opening connection to 44.202.214.112 on port 1228
[x] Opening connection to 44.202.214.112 on port 1228: Trying 44.202.214.112
[+] Opening connection to 44.202.214.112 on port 1228: Done


#### Read from remote socket

In [4]:
recieved = conn.recvline()

while recieved != (b'\n'):
    print(f"{recieved}")
    recieved = conn.recvline()


b'You must solve 1000 of these math problems that are outputted in the following format {number} {operation} {number} to get the flag. \n'
b'Division is integer division using the // operator. \n'
b'The input is being checked through python input() function. \n'
b'Good luck! \n'


#### Write to remote socket

In [5]:
math_problem = conn.recvline(timeout=5)
print(f"current problem is: {math_problem}")

current problem is: b'40 * 74\n'


#### A function to parse the next line

In [6]:
math_problem_ASCII = math_problem.decode('ASCII')
print(f"current problem is: {math_problem_ASCII.split(' ')}")

current problem is: ['40', '*', '74\n']


#### A function to perform calculations

In [7]:
def perform_calc (input: list) -> int:
    input[0] = int(input[0])
    input[2] = int(input[2])
    if input[1] == '+':
        return input[0]+input[2]
    if input[1] == '-':
        return input[0]-input[2]
    if input[1] == '*':
        return input[0]*input[2]     
    if input[1] == '//':
        return input[0]//input[2]     

current_solution = perform_calc(math_problem_ASCII.split(' '))
print(current_solution)


2960


#### A function to automate the responses

In [8]:
print(conn.sendline(str(current_solution).encode()))
recieved = conn.recvline()
print(recieved)
count = 1

None
b'Correct!\n'


In [9]:
while count < 1000:
    math_problem = conn.recvline()
    math_problem_ASCII = math_problem.decode('ASCII')
    current_solution = perform_calc(math_problem_ASCII.split(' '))
    conn.sendline(str(current_solution).encode())
    conn.recvline()
    count += 1
    print(f'count: {count}')

count: 2
count: 3
count: 4
count: 5
count: 6
count: 7
count: 8
count: 9
count: 10
count: 11
count: 12
count: 13
count: 14
count: 15
count: 16
count: 17
count: 18
count: 19
count: 20
count: 21
count: 22
count: 23
count: 24
count: 25
count: 26
count: 27
count: 28
count: 29
count: 30
count: 31
count: 32
count: 33
count: 34
count: 35
count: 36
count: 37
count: 38
count: 39
count: 40
count: 41
count: 42
count: 43
count: 44
count: 45
count: 46
count: 47
count: 48
count: 49
count: 50
count: 51
count: 52
count: 53
count: 54
count: 55
count: 56
count: 57
count: 58
count: 59
count: 60
count: 61
count: 62
count: 63
count: 64
count: 65
count: 66
count: 67
count: 68
count: 69
count: 70
count: 71
count: 72
count: 73
count: 74
count: 75
count: 76
count: 77
count: 78
count: 79
count: 80
count: 81
count: 82
count: 83
count: 84
count: 85
count: 86
count: 87
count: 88
count: 89
count: 90
count: 91
count: 92
count: 93
count: 94
count: 95
count: 96
count: 97
count: 98
count: 99
count: 100
count: 101
count:

EOFError: 

In [ ]:
print(conn.recvline())

In [ ]:
conn.close()

#### Try it yourself...
This challenge was originally seen during the [2022 UMASS CTF](https://ctftime.org/event/1561/) event. You can find the original source code here:

https://github.com/UMassCybersecurity/UMassCTF-2022-challenges/tree/main/misc/quickmaths/src



##### Run locally with Docker

[Docker](https://www.docker.com/products/docker-desktop/) is virtualization software for running apps in a sandboxed environment on your host machine.

After installing the docker daemon, run these commands from the folder containing the `Dockerfile`.

```
docker build -t quickmaths .
docker run --name quickmaths --rm -d -p 1228:1228 quickmaths
```
`-t` will create a new container image with the given name.

`--name` can be used to give the running container a name instead of a randomly generated name.

`--rm` will delete the container when it is stopped.

`-d` will start the container as a background process.

`-p` will map the container's port to your localhost.

##### Stop the container and delete the image
```
docker stop quickmaths
docker rmi quickmaths
```
